In [4]:
#!/usr/bin/env python3
# -*- coding: utf-8 -*-
"""
Created on Mon May 14 11:56:35 2018

@author: Eduard

Ce script sert à parser les messages texte des Tweets ayant été scrapés incorporant
l'un de nos termes cible. Ils sont tous concatennés dans un fichier CSV pour lecture
aisée sur Excel et labélisation manuelle.

"""
import json, glob, os
import pandas as pd


# On indique le chemin vers les fichiers JSON de Tweets scrappés
path = os.getcwd()+'/StreamListenerTweepy/Streams/*.jsonl'


# On crée deux fonctions pour itérer le parse sur plusieurs fichiers JSON 
# placés sur le même dossier
def read_json_files(path_to_file):
    with open(path_to_file) as p:
        data = pd.read_json(p, lines=True)
    return data

def giant_list(json_files):
    data_list = []
    for f in json_files:
        data_list.append(read_json_files(f))
    return data_list


# Éxecution de fonctions et chargement de JSON dans un DF concatenné
event_files = glob.glob(path)
tweets = pd.concat(giant_list(event_files), ignore_index=True)
tweets = tweets[["extended_tweet", "retweeted_status"]]
tweets_copy = tweets.copy()


# Extraction des index des lignes non vides et à parser
index_ext = tweets.extended_tweet[tweets.extended_tweet.notnull()].index
index_ret = tweets.retweeted_status[tweets.retweeted_status.notnull()].index


# On remplace les tweets originaux et trunqués par le texte en entier
for i in index_ext:
    tweets["extended_tweet"][i] = tweets.extended_tweet[i]["full_text"]

    
# Pour les messages retweetés (reteewted_status) on a streamé un dict à plusiers niveaux
# Chaque json a une structure différente. Nous voulons récupérer deux champs:
# 1. les messages retweetés en entier et 2. les messages cités par le même usager


# 1. Si le stream a un champ "extended_tweet" on récupère les index des ces tweets
# pour effectuer ensuite une boucle d'extraction du texte en entier.
# Sinon (else), le message n'a pas été trunqué, on utilise le champ entier "text"
j=[]
for i in index_ret:
    if "extended_tweet" in tweets.retweeted_status[i]:
        j.append(i)
    else:
        tweets.retweeted_status[i] = tweets.retweeted_status[i]["text"]

for i in j:
    tweets.retweeted_status[i] = tweets.retweeted_status[i]["extended_tweet"]["full_text"]

    
# 2. Sur la même colonne d'avant on trouve le texte des messages cités par l'usager.  
# On l'a écrasée lors de l'opération précédente, on utilise une copie à la place.
# Pour ne pas écraser la colonne sur 'tweets', on crée une nouvelle colonne 
# ou l'on placera le texte en entier
k=[]
for i in index_ret:
    if "quoted_status" in tweets_copy.retweeted_status[i]:
        k.append(i)
    else:
        tweets["cite_status"] = None

try:
    for i in k:
        tweets.cite_status[i] = tweets_copy.retweeted_status[i]["quoted_status"]["extended_tweet"]["full_text"]
except:
    pass


# Enfin, on fusionne tous les tweets qui doivent être labelisées manuellement et on efface les dupliqués
tweets_dilcrah_keywords = tweets.stack().dropna().drop_duplicates().reset_index(drop=True)


# À cette fin, on exporte en csv pour pouvoir faire la lecture et labélisation sur Excel
tweets_dilcrah_keywords.to_csv('TweetsToLabelCSV/tweets_dilcrah_keywords.csv', sep=',', encoding='utf-8', header = False)


tweets_dilcrah_keywords

0      @iYume_ @OhmArsene @francoiscarpit1 @PorcherTh...
1      L’objectif de ce post était simple : quelle la...
2      Propos honteux et nauséabonds... Quel est l'ob...
3      Donc la y’a tellement 3 white qui veulent m’ex...
4      @YassEncore LAISSEZ LES ENFANTS CHOISIR DE LEU...
5      Depuis qu'elle a répondu à une itw sur la séle...
6      Tes une babtou tu peux pas comprendre https://...
7      Athée, je ne pourrais pas me sentir représenté...
8      Juste pour bien recontextualiser les choses : ...
9      il ya un mec qui ma mentionné alors que  sa go...
10     «beurette» c’est surtout un mot dégradant aux ...
11     @Sakwws C'est pas mon problème. Arrêtez de ple...
12     L'@UNEF décide d'afficher le vêtement le + rac...
13     @CISWhiteMale4 @Arkhanciel @Nathandco @lordmah...
14     Amazon fait venir des travailleurs étrangers p...
15     Si vous voyez quelqu’un avec ce type de lunett...
16     C'est grave vrai, les mecs de la bac aussi ils...
17     Vous imaginez une petite